# Alpha Separation
We have a picture of a piece of metal. In the image are two types of alpha crystal - Primary alpha are round large blobs and secondary alpha is the smaller needle like crystals. The objective is to create an image mask to separate the two types of crystal

In [ ]:
from skimage import io, color, measure, morphology
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed
import numpy as np
import scipy
from tqdm.notebook import tqdm

plt.rcParams['figure.figsize'] = [18, 12]

## Import image
It's stored as RGBA so convert to greyscale

In [ ]:
def plot_gray(image: np.ndarray):
    """Plot the supplied array as an image in greyscale."""
    io.imshow(image, cmap="gray")
    plt.show()
    
def plot_color(image: np.ndarray):
    """Plot the supplied array as an image using the standard colormap."""
    io.imshow(image)
    plt.show()

In [ ]:
image = io.imread("BSE 200um FW.png")
gray_image = color.rgb2gray(color.rgba2rgb(image))
plot_gray(gray_image)

## Crop image
Remove white borders

In [ ]:
x_min = 0
x_max = 1093
y_min = 21
y_max = 843
gray_image = gray_image[y_min:y_max, x_min:x_max]
plot_gray(gray_image)

## What is interesting to the human eye about this image?
This will inform our approach for the analysis.

One notable aspect is that the primary alpha tends to be more oval while the rods have a much higher aspect ratio. There are some high aspect ratio primary alpha sections, but these are on a much larger scale. On average the primary alpha regions are bigger than the secondary alpha.

There is some white background that is neither primary nor secondary. All secondary is close to some white background, but the center of the alpha is not close to any white.

## Attempt 1: Characterise distance of pixels from white background
- First threshold the image to get the white particles and non-white particles
- Then for all non-white pixels, measure the average distance to the nearest half dozen white pixels

### Filter image
Setting the threshold slightly on the low side is good here. Some noise in the primary alpha is probably better than losing definition from the background.

In [ ]:
def threshold_image_display(image: np.ndarray, threshold: int):
    print(image.shape)
    mask = (image > threshold).astype(int)
    plot_gray(mask)
    print(f"{image.size} total pixels")
    print(f"{np.sum(mask)} background pixels")

In [ ]:
interact(threshold_image_display, image=fixed(gray_image), threshold=widgets.FloatSlider(min=0, max=1, step=1/256, value=0.45, continuous_update=False))

A threshold of 0.45 pretty much separates the alpha and the background.

### Measure distance from each pixel to the background 
Select the background pixels using the threshold.

In [ ]:
# Have to re-stack the indices here as np.where returns the columns and rows separately.
background_pixel_locations = np.column_stack(np.where(gray_image > 0.45))

Now measure the distance between each particle in the image and its N nearest background particles (background has value of 1)

For testing purposes it is best to set the x_lim and y_lim to do a small subregion. A 200x200 section takes on the order of 1 minute to run.

In [ ]:
num_pixels = 10

x_lim = 200
y_lim = 200
new_image = []
indices = np.indices((1, y_lim)).reshape(2, y_lim).T
for x in tqdm(range(x_lim)):
    # Do cdist in sqeuclidean and sqrt the end result to save time.
    distances = scipy.spatial.distance.cdist(indices, background_pixel_locations, 'sqeuclidean')
    # Get the indices of the N closest background pixels
    shortest_indices = np.argpartition(distances, num_pixels, axis=1)[:, :num_pixels]
    # Get the distances of the N closest background pixels using the indices.
    # Indexing a 2D array with a 2D array is weird... no idea why it is this hard.
    distance_to_background = np.mean(distances[np.arange(distances.shape[0])[:, None], shortest_indices], axis=1)
    new_image.append(distance_to_background)
    indices[:, 0] += 1

In [ ]:
distance = np.sqrt(np.array(new_image).reshape(x_lim, y_lim))
plot_color(distance)

Now threshold again - primary alpha is the higher values.

In [ ]:
threshold = 8

primary_alpha = distance.copy()
primary_alpha_indices = np.column_stack(np.where(primary_alpha > threshold))
plt.imshow(gray_image, cmap='gray')
plt.scatter(primary_alpha_indices[:, 1], primary_alpha_indices[:, 0], s=3)
plt.show()

This has worked pretty well. It is picking up just a touch of secondary alpha at (80,180) but this could be removed by labelling image regions and filtering on size. 

The marked region needs to be expanded a little to capture the full alpha region. This can be done by applying a circle brush to each identified point. This can be easily simulated by changing the scatter plot points to be bigger - a brush of about 30px is about right.

This algorithm is also pretty slow - about 20 minutes for this sample image. 30% of this is due to argparse but this is needed to prevent noise disrupting the signal (perhaps some filtering would preclude this). The rest of the time is in cdist, and I don't think there is any way to game this. My basic implementation of a cell list turns out slower because it isn't vectorised. It would be lighting fast compiled as C with a cell list but meh... too much effort.

# Attempt 2 - A simpler filter
What about just trying to threshold and then dilate?

First we must do some denoising of the thresholded image. A basic gaussian blur strong enough to bring salt and pepper noise below the threshold would greatly reduce the crispness of the background - it is too blunt an instrument. Mostly we want to just get the single white pixels in the black regions as these will cause problems with the dilation. We can do this by considering the neighbourhood of the noise particles. Any white pixels mostly surrounded by black pixels are considered noise. Even with this method, the thresholds cannot be set too agressively as this blurs the edges between the alpha and the background.

The result shouldn't look much different - it should just be the odd speckle from the big primary alpha sections that is removed.

In [ ]:
binary = (gray_image > 0.48).astype(int)
plot_gray(binary)

mask_size = 4
threshold_pixels = 5

for x in range(mask_size, binary.shape[0] - mask_size):
    for y in range(mask_size, binary.shape[1] - mask_size):
        if binary[x, y] == 1:
            if np.sum(binary[x - mask_size:x + mask_size, y - mask_size:y + mask_size]) < threshold_pixels:
                binary[x, y] = 0

plot_gray(binary)

Now try a dilation x times. Too much dilation is bad as it will dilate the structures we want to keep. Too little dilation and the primary and secondary regions will still be contiguous. About 6 or 7 seems good for this image.

In [ ]:
dilated_binary = binary.copy()

for x in range(7):
    dilated_binary = morphology.dilation(dilated_binary)
plot_gray(dilated_binary)

Now the regions are seperate, label contiguous regions. Labelling will label white regions, so the image must be inverted using a logical not. This puts the background as label zero.

In [ ]:
labelled = morphology.label(np.logical_not(dilated_binary))
plot_color(labelled)

Now iterate through the labels, removing small ones.

In [ ]:
threshold = 80

label_props = measure.regionprops(labelled)
for index, label in enumerate(label_props):
    if label["area"] < threshold:
        # Regionprops starts at label 1 so must increment index by 1
        labelled[labelled == index + 1] = 0
plot_color(labelled)

mask = np.column_stack(np.where(labelled != 0))
plt.imshow(gray_image, cmap='gray')
plt.scatter(mask[:, 1], mask[:, 0], s=3, alpha=0.1)
plt.show()

Pretty good! The repeated dilations have removed some smaller primary alpha particles. It might be better to do fewer dilations and a smarter filtering of the labels - perhaps using the eccentricity or one of the other label properties.

This could be done by manually identifying labels that are desired, and those that are not and clustering their properties - perhaps an opportunity for *sklearn.cluster*. I initially thought it would be too hard to get the secondary and the primary non-contiguous without wiping out all the detail which is why I tried the complicated method first. This one looks similarly robust and runs in seconds.